<a href="https://www.kaggle.com/code/christiandeangelis1/digit-recognizer-with-pytorch?scriptVersionId=117733658" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import torch
import pandas as pd
from torch import nn, optim
import torch.nn.functional as F

# Digit Recognizer

This is a work in progress, I will soon add prose explaining my approach. The code and model is complete, although can be improved.

In [2]:
import torchvision.transforms as transforms

transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize((0.5),(0.5))])

In [3]:
device = torch.device("cuda")

In [4]:
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

X = train.iloc[:,1:].values.astype('float32').reshape(-1,28,28)
y = train.iloc[:,0]

In [5]:
from torch.utils.data import Dataset

class ImgDataset(Dataset):
    def __init__(self, imgs, labels, transform=None):
        self.labels = labels # pd Series
        self.imgs = imgs 
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        if self.transform:
            image = transform(self.imgs[idx])
        return image, label

In [6]:
from torch.utils.data import random_split

train_data = ImgDataset(X,y, transform=True)

size = 0.1
v_size = round(len(train_data)*size)
t_size = len(train_data) - v_size

train_set, valid_set = random_split(train_data, [t_size, v_size])

In [7]:
from torch.utils.data import DataLoader

batch_size=128 

dl = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_set, batch_size=batch_size, shuffle=True)

In [8]:
class CNN(nn.Module):

    def __init__(self, out_1 = 32, out_2 = 64, n_classes = 10):
        super(CNN, self).__init__()
        self.bn1 = nn.BatchNorm2d(32, eps=1e-07, momentum=0.9, affine=True)
        self.bn2 = nn.BatchNorm2d(64, eps=1e-07, momentum=0.9, affine=True)

        self.cnn1 = nn.Conv2d(1, 32, kernel_size = 3, padding = 1)
        self.maxpool1 = nn.MaxPool2d(kernel_size = 2)
        
        self.cnn2 = nn.Conv2d(32, 64, kernel_size = 3, padding = 1)
        self.maxpool2 = nn.MaxPool2d(kernel_size = 2)
        self.fc1 = nn.Linear(out_2 * 7 * 7, n_classes)
        
    def forward(self, x):
        x = self.cnn1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.maxpool1(x)
        x = self.cnn2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return(x)

In [9]:
def train_epoch(model):
    size = len(dl.dataset)
    for batch, (X,y) in enumerate(dl):
        X = X.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def validate_epoch(model):
    size = len(valid_dl.dataset)
    num_batches = len(valid_dl)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for (X, y) in valid_dl:
            X = X.to(device)
            y = y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n") 

In [11]:
def train_model(model,n_epochs):
    model = model.to(device)
    for i in range(n_epochs):
        print(f"Epoch {i+1}\n-------------------------------")
        train_epoch(model)
        validate_epoch(model)
    print("Done.")

In [12]:
lr = 0.002
n_epochs = 10
momentum = 0.9

model = CNN()

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

train_model(model,n_epochs)

Epoch 1
-------------------------------
loss: 2.343206  [    0/42000]
loss: 0.164767  [12800/42000]
loss: 0.170232  [25600/42000]
loss: 0.075171  [38400/42000]
Test Error: 
 Accuracy: 97.4%, Avg loss: 0.098925 

Epoch 2
-------------------------------
loss: 0.090688  [    0/42000]
loss: 0.120247  [12800/42000]
loss: 0.036068  [25600/42000]
loss: 0.074334  [38400/42000]
Test Error: 
 Accuracy: 98.2%, Avg loss: 0.065351 

Epoch 3
-------------------------------
loss: 0.062546  [    0/42000]
loss: 0.050012  [12800/42000]
loss: 0.067335  [25600/42000]
loss: 0.043871  [38400/42000]
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.053255 

Epoch 4
-------------------------------
loss: 0.052555  [    0/42000]
loss: 0.059716  [12800/42000]
loss: 0.043129  [25600/42000]
loss: 0.032203  [38400/42000]
Test Error: 
 Accuracy: 98.8%, Avg loss: 0.049828 

Epoch 5
-------------------------------
loss: 0.036864  [    0/42000]
loss: 0.048866  [12800/42000]
loss: 0.024138  [25600/42000]
loss: 0.042813  [38400

# Testing

In [13]:
testdata = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

testdata = testdata.values.reshape(-1,1,28,28)
testdata = torch.Tensor(testdata)

submission = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

model.to(device)

for i, test in enumerate(DataLoader(testdata, batch_size=1)):
    test = test.to(device)
    model.eval()
    _,pred = torch.max(model(test),1)
    submission["Label"][i] = pred.item()
    

submission.to_csv("submission.csv", index=False)